In [ ]:
# main.py
import asyncio
from asyncio.log import logger
import json
import sys


from src.gold.contexts.gold_parser import GoldParser
from src.common.azure_clients.event_grid_client_manager import EventGridClientManager
from src.common.models.orchestrator_result import OrchestratorResult
from src.common.enums.etl_layers import ETLLayer
from src.common.config.config_manager import ConfigManager
from src.common.spark.spark_service import SparkService
from src.common.factories.orchestrator_factory import OrchestratorFactory
import src.gold.init.gold_init

In [ ]:
def send_event_grid_notification(config: ConfigManager, orchestrator_result: OrchestratorResult):
        """
        Tworzy i wysyła powiadomienie Event Grid na podstawie OrchestratorResult.
        """
        logger.info(f"Sending Event Grid notification for correlation ID: {orchestrator_result.correlation_id}")
        

        event_grid_client = EventGridClientManager(
            endpoint=config.get("EVENT_GRID_ENDPOINT"),
            key=config.get("EVENT_GRID_KEY")
        )
        if not event_grid_client:
            logger.error("Event Grid client not initialized. Cannot send event.")
            return

        payload = {
            "layer": orchestrator_result.etl_layer.value,
            "env": orchestrator_result.env.value,
            "status": orchestrator_result.status,
            "message_date": orchestrator_result.timestamp.isoformat(),
            "correlation_id": orchestrator_result.correlation_id,
            "manifest": "/gold/manifest/dev.manifest.json",
            "summary_processing_uri": orchestrator_result.summary_url,
            "duration_in_ms": orchestrator_result.duration_in_ms
        }

        try:
            event_grid_client.send_event(
                event_type=f"{orchestrator_result.etl_layer.value.capitalize()}ProcessCompleted",
                subject=f"/{orchestrator_result.etl_layer.value}/processing/{orchestrator_result.correlation_id}",
                data=payload
            )
            logger.info("Event Grid notification sent successfully.")
        except Exception as e:
            logger.error(f"Failed to send Event Grid notification: {e}", exc_info=True)
            



In [ ]:
dbutils.widgets.text("summary_json", "")
dbutils.widgets.text("manifest_json", "")

manifest_json = dbutils.widgets.get("manifest_json")
summary_json = dbutils.widgets.get("summary_json")

config = ConfigManager()
context = GoldParser(config).parse(manifest_json=manifest_json, summary_json=summary_json)

spark = SparkService(config)
spark.start_local()


orchestrator = OrchestratorFactory.get_instance(ETLLayer.GOLD, spark=spark, config=config)
result = asyncio.run(orchestrator.execute(context))
send_event_grid_notification(config=config, orchestrator_result=result)